# KoChatGPT 업그레이드 하기

프로젝트 루브릭:
| 학습 목표 | 평가 기준 |
|:----------|:-----------|
| 기존 KoGPT2와 SFT 적용 모델 결과 분석했는가? | 기존 모델의 결과물과 SFT를 적용한 모델의 결과물을 정량/정성적으로 비교/분석했다. |
| SFT 모델과 RM 모델 결과 분석을 해보았는가? | SFT를 적용한 모델의 결과물과 RM을 적용한 모델의 결과물을 정량/정성적으로 비교/분석했다. |
|데이터셋 정제 / 새로운 데이터셋 / foundation model 교체 중 하나를 이용해 정량적 성능 향상을 해보았는가? | 1. 기존 데이터셋을 추가로 정제하고, generation 성능을 올리기 위한 기법(Beam search, Top-k sampling 등)을 실험해 모델 성능을 향상시켰다. <br> 2. 새로운 데이터를 수집해 전처리를 수행하여 모델의 성능을 향상시켰다. <br> 3. 더 적절한 학습 전략(SFT, RM, PPO)을 적용하거나 initial model을 변경해 모델의 성능을 향상시켰다.|

# Section 1. 라이브러리 설치 & 로드

In [2]:
!pip install datasets
!pip install loralib
!pip install trl
!pip install accelerate
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 112.3 MB/s eta 0:00:0000:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 20.0.0
    Uninstalling pyarrow-20.0.0:
      Successfully uninstalled pyarrow-20.0.0━━━━━━━━━━━━━━━━━━━━━  3/15 [pyarrow]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15/15 [datasets]/15 [datasets]ce-hub]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 46.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface_hub 1.2.1
    Uninstalling huggingface_hub-1.2.1:━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/

In [1]:
import os
import json
import torch
import torch.nn as nn
from typing import Dict, Sequence
from torch.utils.data import Dataset
from copy import deepcopy
from dataclasses import dataclass
import pandas as pd
import gc
import transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    Trainer, 
    TrainingArguments, 
    DataCollatorForLanguageModeling
)
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# KoChatGPT 모듈 임포트
try:
    from chatgpt.dataset import RewardDataset
    from chatgpt.models.base import RewardModel
    from chatgpt.models.gpt import GPTActor, GPTCritic
    from chatgpt.trainer import PPOTrainer, RewardModelTrainer
    from chatgpt.trainer.strategies import NaiveStrategy
except ImportError:
    print("⚠️ 'chatgpt' 모듈을 찾을 수 없습니다. KoChatGPT 폴더가 있는 위치에서 실행해주세요.")

# Section 2. 환경 설정

In [3]:
# 1. Device 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 2. Model ID 변수화
BASE_MODEL_ID = 'skt/kogpt2-base-v2'

# 3. 하이퍼파라미터
BATCH_SIZE = 4
GRAD_ACCUMULATION = 2
SFT_EPOCHS = 1
RM_EPOCHS = 1
PPO_EPOCHS = 1

# 4. 프롬프트 템플릿
PROMPT_DICT = {
    "prompt_input": "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
}

# 5. 토크나이저 로드 (Global)
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL_ID, 
    padding_side="right", 
    model_max_length=512
)
# KoGPT2는 pad_token이 없으므로 eos_token을 pad_token으로 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 6. 평가용 프롬프트
eval_prompts = [
    "불고기용 고기 한우에요?",
    "리처드 닉슨이 43대 부통령직을 수행한 년도는?",
    "시카고 오헤어 국제공항은 어디에 있어?",
    "오늘 미세먼지 어때?",
]

# 7. 결과 저장소
final_report = {"Prompt": eval_prompts}

def generate_beam(model, prompt):
    model.eval()
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_length=128,
            num_beams=5,              # Beam Search 적용
            no_repeat_ngram_size=3,   # 반복 방지
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    gen_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 프롬프트 제거 및 답변 추출
    try: response = gen_text.split("### Response(응답):")[1].strip()
    except: response = gen_text
    return response

# 8. RM 점수 계산 함수 (정량 평가용)
def get_score(rm_model, text):
    if rm_model is None: return 0.0
    rm_model.eval()
    input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
    with torch.no_grad():
        reward = rm_model(input_ids)
    return reward.item()

Using device: cuda


# Section 3. Baseline (KoGPT-2) 평가

In [4]:
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_ID).to(device)
# [Fix] 모델의 Pad Token ID도 동기화
base_model.config.pad_token_id = tokenizer.pad_token_id

base_res = []
print("Generating Baseline responses...")
for p in eval_prompts:
    res = generate_beam(base_model, p)
    base_res.append(res)
    print(f"[Base] {res}")

final_report['Base_Response'] = base_res
del base_model
torch.cuda.empty_cache()

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generating Baseline responses...
[Base] 불고기용 고기 한우에요? ᄏᄏᄏ
이거 진짜 맛있었어요!!!!
고기랑 같이 먹으니까 더 맛있더라구요 ᄒᄒᄒ
고기랑 함께 먹으면 더 맛있어요!!
이렇게 고기랑 같이 먹으면 더 맛있는 것 같아요  ᅲ
이렇게 먹어도 맛있을 것 같은데...
이렇게 맛있게 잘 먹었습니다!! #20180413 #미롱_식단 
점심 : #다히먹방
#diet
[Base] 리처드 닉슨이 43대 부통령직을 수행한 년도는?"
"그렇지 않습니다."
"아니오, 아니오."
"그럼, 닉슨이 부통령직을 수행했군요."
"이봐, 닉슨 씨?"
닉슨은 고개를 끄덕였다.
"아니, 아니오. 닉슨 씨가 부통령직을 맡았군요!"
"네, 그렇습니다."
닉슨이 말했다.
"그런데 그게 무슨 뜻입니까?"
그제야 닉슨이 대답했다.
"닉슨 씨는 부통령직 수행 중입니다."
"그래, 그렇군요. 닉슨은 부통령직을 수행하고 있습니다."
"어떻게 된 일입
[Base] 시카고 오헤어 국제공항은 어디에 있어?"
"어디서 왔어요?"
그녀는 고개를 끄덕였다.
"아무것도 묻지 않았어요."
"그런데 무슨 일이에요? 무슨 일이야?"
이번에도 그녀는 대답하지 않았다.
"어떻게 된 거예요? 어젯밤에 무슨 일이 있었어요!"
"아니, 그게 무슨 말이에요! 무슨 일이 있었던 거예요."
그녀의 목소리는 더 이상 들리지 않았다.
"그게 무슨 말씀이세요? 아, 그게 뭐예요! 어젯밤 무슨 일이었죠?"
그러자
[Base] 오늘 미세먼지 어때? 아~ 예. 예. 미세먼지가 어때요?
아~ 예예. 아~ 미세먼지는 어때요.
예. 어~ 아~ 아까 말씀드린 것처럼 예. 그~ 대기 중에 미세먼지를 걸러주는 역할을 하기 때문에 예. 이 미세먼지에 대한 걱정은 하지 않아도 될 것 같습니다.
예. 자 오늘은 미세먼지와 관련된 여러 가지 이야기 나눠보도록 하겠습니다.
먼저 미세먼지의 원인부터 알아보겠습니다.
네. 미세먼지로 인한 호흡기 질환에 대해서 알아봅니다.
먼저 호흡기 질환으로 인한 기관지 천식 그렇습니다.
어~ 기관

# Section 4. SFT
Supervised Fine-Tuning

In [5]:
class SFT_Dataset_Best(Dataset):
    def __init__(self, file_path, tokenizer):
        self.input_ids = []
        self.labels = []
        with open(file_path, "r", encoding='utf-8-sig') as f:
            data = json.load(f)
        print(f"SFT Data Size: {len(data)}")

        sources = []
        targets = []
        for item in data:
            sources.append(PROMPT_DICT["prompt_input"].format_map({"prompt": item['prompt']}))
            targets.append(item['completion'] + tokenizer.eos_token)
            
        examples = [s + t for s, t in zip(sources, targets)]
        
        sources_tokenized = tokenizer(sources, padding=False, truncation=True, max_length=512)
        examples_tokenized = tokenizer(examples, padding=False, truncation=True, max_length=512)
        
        for input_id, source_len in zip(examples_tokenized["input_ids"], [len(l) for l in sources_tokenized["input_ids"]]):
            self.input_ids.append(torch.tensor(input_id, dtype=torch.long))
            label = torch.tensor(input_id, dtype=torch.long)
            mask_len = min(source_len, len(input_id))
            label[:mask_len] = -100
            self.labels.append(label)

    def __len__(self): return len(self.input_ids)
    def __getitem__(self, idx): return dict(input_ids=self.input_ids[idx], labels=self.labels[idx])

@dataclass
class DataCollatorForSupervisedDataset(object):
    tokenizer: transformers.PreTrainedTokenizer
    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)
        return dict(input_ids=input_ids, labels=labels, attention_mask=input_ids.ne(self.tokenizer.pad_token_id))

sft_dataset = SFT_Dataset_Best('KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_ID).to(device)

# [Fix] 핵심 수정: 토크나이저 크기에 맞춰 모델 임베딩 사이즈 조정 (에러 해결)
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

training_args = TrainingArguments(
    output_dir="sft_output", overwrite_output_dir=True, num_train_epochs=SFT_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE, gradient_accumulation_steps=GRAD_ACCUMULATION,
    save_strategy="no", logging_steps=100, 
    fp16=True, # CUDA 에러가 계속되면 False로 변경 고려
    gradient_checkpointing=True
)

trainer = Trainer(model=model, args=training_args, train_dataset=sft_dataset, data_collator=data_collator)
trainer.train()

sft_res = []
print("Generating SFT responses...")
for p in eval_prompts:
    p_fmt = PROMPT_DICT["prompt_input"].format_map({"prompt": p})
    sft_res.append(generate_beam(model, p_fmt))

final_report['SFT_Response'] = sft_res
model.save_pretrained('models/output_1_SFT')
del model, trainer
torch.cuda.empty_cache()

SFT Data Size: 12000


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,3.390200
200,3.039600
300,2.923600
400,2.894800
500,2.836500
600,2.839500
700,2.824600
800,2.783600
900,2.768900
1000,2.698200


Generating SFT responses...


# Section 5. RM

In [8]:
class StaticRewardDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_len=512):
        self.data = []
        with open(file_path, "r", encoding='utf-8-sig') as f:
            raw_data = json.load(f)
        for item in raw_data:
            p, r = item['prompt'], item['ranking']
            # 3배수 증강
            for i, j in [(0,1), (0,2), (1,2)]:
                if r[i] < r[j]:
                    self.data.append({'prompt': p, 'chosen': item[f'completion_{i}'], 'rejected': item[f'completion_{j}']})
                else:
                    self.data.append({'prompt': p, 'chosen': item[f'completion_{j}'], 'rejected': item[f'completion_{i}']})
        
        print(f"RM Data Size (Augmented): {len(self.data)}")
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self): return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        
        # [Fix] padding='max_length'로 정적 패딩 수행
        # Trainer 내부에서 squeeze(1)을 하므로 [0] 인덱싱을 하지 않고 (1, len) shape 유지
        inputs_c = self.tokenizer(
            item['prompt'], item['chosen'], 
            return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_len
        )
        inputs_r = self.tokenizer(
            item['prompt'], item['rejected'], 
            return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_len
        )
        
        # [Fix] Dictionary 대신 Tuple 반환 (순서 중요: chosen -> rejected)
        # Trainer 코드: for chosen_ids, c_mask, reject_ids, r_mask in self.train_dataloader:
        return (
            inputs_c['input_ids'],       # chosen_ids
            inputs_c['attention_mask'],  # c_mask
            inputs_r['input_ids'],       # reject_ids
            inputs_r['attention_mask']   # r_mask
        )

# RM 모델 클래스 (기존 동일)
class GPTRM_custom(RewardModel):
    def __init__(self, pretrained=None):
        model = GPT2Model.from_pretrained(pretrained)
        model.resize_token_embeddings(len(tokenizer))
        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head)

# 학습 준비
rm_model = GPTRM_custom(pretrained='models/output_1_SFT').cuda()
rm_dataset = StaticRewardDataset('KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl', tokenizer)

# Trainer 초기화 (data_collator 제거됨)
rm_trainer = RewardModelTrainer(
    model=rm_model,
    strategy=NaiveStrategy(),
    optim=torch.optim.Adam(rm_model.parameters(), lr=5e-5),
    train_dataset=rm_dataset,
    eval_dataset=None,
    batch_size=BATCH_SIZE, 
    max_epochs=RM_EPOCHS
)

# 학습 시작
rm_trainer.fit(use_lora=0)
rm_model.save_pretrained('models/output_2_RM')

# 중간 점수 측정
print("Scoring responses with RM...")
final_report['Base_Reward'] = [get_score(rm_model, r) for r in final_report['Base_Response']]
final_report['SFT_Reward'] = [get_score(rm_model, r) for r in final_report['SFT_Response']]

# 메모리 정리
del rm_model, rm_trainer
torch.cuda.empty_cache()

RM Data Size (Augmented): 30660


Train epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Train step of epoch 0:   0%|          | 0/7665 [00:00<?, ?it/s]

TypeError: object of type 'NoneType' has no len()

# Section 6. PPO

In [ ]:
with open('KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as f:
    ppo_data = json.load(f)
    ppo_prompts = [d['prompt'] for d in ppo_data]

def ppo_tokenize(texts):
    batch = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    return {k: v.cuda() for k, v in batch.items()}

with NaiveStrategy().model_init_context():
    # [Fix] 모든 모델 로드 시 resize_token_embeddings 호출이 보장되어야 함 (저장된 모델은 이미 적용됨)
    actor = GPTActor(pretrained='models/output_1_SFT', lora_rank=0).to(device)
    critic = GPTCritic(pretrained='models/output_2_RM', lora_rank=0).to(device)
    initial_model = deepcopy(actor).to(device)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(device)

actor_optim = torch.optim.Adam(actor.parameters(), lr=5e-6)
critic_optim = torch.optim.Adam(critic.parameters(), lr=5e-6)

ppo_trainer = PPOTrainer(
    NaiveStrategy(), actor, critic, reward_model, initial_model,
    actor_optim, critic_optim, max_epochs=PPO_EPOCHS, 
    train_batch_size=BATCH_SIZE, gradient_accumulation_steps=GRAD_ACCUMULATION,
    tokenizer=ppo_tokenize, do_sample=True, max_length=128,
    pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id
)

ppo_trainer.fit(ppo_prompts, num_episodes=10, max_timesteps=3, update_timesteps=3)

ppo_res = []
ppo_scores = []
print("Generating PPO responses...")
for p in eval_prompts:
    p_fmt = PROMPT_DICT["prompt_input"].format_map({"prompt": p})
    inputs = tokenizer(p_fmt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = actor.model.generate(
            inputs.input_ids, max_length=128, num_beams=5, no_repeat_ngram_size=3,
            early_stopping=True, pad_token_id=tokenizer.pad_token_id
        )
    res = tokenizer.decode(outputs[0], skip_special_tokens=True)
    try: res_only = res.split("### Response(응답):")[1].strip()
    except: res_only = res
    ppo_res.append(res_only)
    ppo_scores.append(get_score(reward_model, res))

final_report['PPO_Response'] = ppo_res
final_report['PPO_Reward'] = ppo_scores
actor.model.save_pretrained('models/output_3_PPO')

# Section 7. 결과

In [ ]:
df = pd.DataFrame(final_report)
pd.set_option('display.max_colwidth', None)
display(df)

print("\n>>> [정량적 분석] 평균 보상 점수 (Reward Score)")
print(f"Base Model: {df['Base_Reward'].mean():.4f}")
print(f"SFT Model : {df['SFT_Reward'].mean():.4f}")
print(f"PPO Model : {df['PPO_Reward'].mean():.4f}")